In [1]:
import os
local_dev =  "/workspace/BYU/notebooks" if "WANDB_API_KEY" in os.environ else "C:/Users/Freedomkwok2022/ML_Learn/BYU/notebooks"
yolo_dataset_dir = os.path.join(local_dev, 'yolo_dataset')
yolo_weights_dir = os.path.join(local_dev, 'yolo_weights')

In [ ]:
import torch
from ultralytics import YOLO
model_009_merge_159 = os.path.join(yolo_weights_dir, "159_yolo11m_shared_009_merged_120_150")
model = YOLO(model_009_merge_159)

# Dictionary to store activations
activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

import matplotlib.pyplot as plt

def plot_feature_maps(feature_map, num_channels=8):
    fig, axes = plt.subplots(1, num_channels, figsize=(20, 5))
    for i in range(num_channels):
        axes[i].imshow(feature_map[0, i].cpu(), cmap='viridis')
        axes[i].axis('off')
    plt.show()

# Register hook to the desired layer
# Adjust the index based on the model's architecture
model.model.model[0].register_forward_hook(get_activation('layer0'))

### Visualize the Feature Maps

In [ ]:

import cv2

# Load and preprocess the image
img = cv2.imread('path_to_image.jpg')
results = model(img)

# Run inference
model.predict(source=img)
# Example: Visualize the first 8 channels from 'layer0'
plot_feature_maps(activations['layer0'], num_channels=8)

In [ ]:
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image
from matplotlib.cm import get_cmap

# Load YOLO11 model
model = YOLO('yolo11s.pt')  # Or 'yolo11m.pt'
model.eval()

# Target layer (you can try other indices, e.g., 5, 10, 15)
target_layer_idx = 10

# Store feature maps and gradients
gradients = []
activations = []

def forward_hook(module, input, output):
    activations.append(output)

def backward_hook(module, grad_in, grad_out):
    gradients.append(grad_out[0])

# Register hooks
target_layer = model.model.model[target_layer_idx]
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

# Load and preprocess image
img_path = 'path_to_your_image.jpg'
img = Image.open(img_path).convert("L") 
preprocess = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
input_tensor = preprocess(img).unsqueeze(0)

# Forward pass
output = model(input_tensor)[0]
score = output[:, 4].max()  # objectness confidence
model.zero_grad()
score.backward()

# Generate Grad-CAM
grad = gradients[-1].squeeze(0)
act = activations[-1].squeeze(0)
weights = grad.mean(dim=(1, 2), keepdim=True)
cam = (weights * act).sum(dim=0)
cam = torch.relu(cam)
cam -= cam.min()
cam /= cam.max()
cam = cam.detach().cpu().numpy()

# Resize CAM and overlay
cam_resized = cv2.resize(cam, img.size)
heatmap = get_cmap('jet')(cam_resized)[:, :, :3]
heatmap = (heatmap * 255).astype(np.uint8)
overlay = np.array(img) * 0.5 + heatmap * 0.5
overlay = overlay.astype(np.uint8)

# Show result
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(overlay)
plt.title("Grad-CAM Overlay")
plt.axis("off")
plt.tight_layout()
plt.show()